In [3]:
import pygame
import heapq
import time

# Grid size
WIDTH, HEIGHT = 600, 600
ROWS, COLS = 20, 20
CELL_SIZE = WIDTH // COLS

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)

# Initialize pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("A* Pathfinding Visualization")

# Grid representation
grid = [[0 for _ in range(COLS)] for _ in range(ROWS)]
start, goal = None, None

# Directions for movement
MOVES = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def draw_grid():
    for y in range(ROWS):
        for x in range(COLS):
            color = WHITE
            if grid[y][x] == 1:
                color = BLACK
            elif (y, x) == start:
                color = GREEN
            elif (y, x) == goal:
                color = RED
            pygame.draw.rect(screen, color, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
            pygame.draw.rect(screen, BLACK, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)

def astar(start, goal):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    visited = set()
    
    while open_set:
        pygame.event.pump()
        _, current = heapq.heappop(open_set)
        visited.add(current)
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]
        
        for move in MOVES:
            neighbor = (current[0] + move[0], current[1] + move[1])
            
            if 0 <= neighbor[0] < ROWS and 0 <= neighbor[1] < COLS and grid[neighbor[0]][neighbor[1]] == 0:
                temp_g_score = g_score[current] + 1
                
                if neighbor not in g_score or temp_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = temp_g_score
                    f_score[neighbor] = temp_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
                    
                    pygame.draw.rect(screen, YELLOW, (neighbor[1] * CELL_SIZE, neighbor[0] * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                    pygame.display.flip()
                    time.sleep(0.05)
    return None

def main():
    global start, goal
    running = True
    path = None
    
    while running:
        screen.fill(WHITE)
        draw_grid()
        
        if path:
            for p in path:
                pygame.draw.rect(screen, BLUE, (p[1] * CELL_SIZE, p[0] * CELL_SIZE, CELL_SIZE, CELL_SIZE))
        
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            elif pygame.mouse.get_pressed()[0]:  # Left click for obstacles
                x, y = pygame.mouse.get_pos()
                grid[y // CELL_SIZE][x // CELL_SIZE] = 1
            
            elif pygame.mouse.get_pressed()[2]:  # Right click for start/goal
                x, y = pygame.mouse.get_pos()
                cell = (y // CELL_SIZE, x // CELL_SIZE)
                if not start:
                    start = cell
                elif not goal:
                    goal = cell
            
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start and goal:
                    path = astar(start, goal)
                elif event.key == pygame.K_c:  # Clear grid
                    grid[:] = [[0 for _ in range(COLS)] for _ in range(ROWS)]
                    start, goal, path = None, None, None
    
    pygame.quit()

main()
